In [1]:
import pandas as pd
import numpy as np

In [2]:
import ngram
import pickle
from helpers import *
from CountFeatureGenerator import *
from TfidfFeatureGenerator import *
from SvdFeatureGenerator import *
from Word2VecFeatureGenerator import *
from SentimentFeatureGenerator import *
#from AlignmentFeatureGenerator import *


In [3]:
body_train = pd.read_csv("../datasets/train_bodies_processed.csv", encoding='utf-8')

In [4]:
body_train.head(10)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...
5,8,There is so much fake stuff on the Internet in...
6,9,"(CNN) -- A meteorite crashed down in Managua, ..."
7,10,"Move over, Netflix and Hulu.\nWord has it that..."
8,11,We’ve all seen the traditional depictions of G...
9,13,A SOLDIER has been shot at Canada’s National W...


In [5]:
stances_train = pd.read_csv("../datasets/train_stances_processed.csv", encoding='utf-8')

In [6]:
stances_train.head(10)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,agree
6,Accused Boston Marathon Bomber Severely Injure...,962,unrelated
7,Identity of ISIS terrorist known as 'Jihadi Jo...,2033,unrelated
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree
9,British Aid Worker Confirmed Murdered By ISIS,882,unrelated


In [7]:
train = pd.merge(stances_train, body_train, how='left', on='Body ID')

In [8]:
train.head(10)

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's..."
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,agree,Thousands of people have been duped by a fake ...
6,Accused Boston Marathon Bomber Severely Injure...,962,unrelated,A British fighter who travelled to Iraq to sto...
7,Identity of ISIS terrorist known as 'Jihadi Jo...,2033,unrelated,"Adding to Apple's iOS 8 launch troubles, a rep..."
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree,If you’ve seen a story floating around on your...
9,British Aid Worker Confirmed Murdered By ISIS,882,unrelated,The British Islamic State militant who has fea...


In [9]:
targets = ['agree', 'disagree', 'discuss', 'unrelated']

In [10]:
targets_dict = dict(zip(targets, range(len(targets))))

In [11]:
targets_dict

{'agree': 0, 'disagree': 1, 'discuss': 2, 'unrelated': 3}

In [12]:
train['target'] = list(map(lambda x: targets_dict[x], train['Stance']))

In [13]:
train.shape

(49972, 5)

In [14]:
train.head(10)

,Headline,Body ID,Stance,articleBody,target
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...,0
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...,3
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...,3
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's...",1
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,agree,Thousands of people have been duped by a fake ...,0
6,Accused Boston Marathon Bomber Severely Injure...,962,unrelated,A British fighter who travelled to Iraq to sto...,3
7,Identity of ISIS terrorist known as 'Jihadi Jo...,2033,unrelated,"Adding to Apple's iOS 8 launch troubles, a rep...",3
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree,If you’ve seen a story floating around on your...,0
9,British Aid Worker Confirmed Murdered By ISIS,882,unrelated,The British Islamic State militant who has fea...,3


In [15]:
body_test = pd.read_csv("../datasets/test_bodies_processed.csv", encoding='utf-8')
headline_test = pd.read_csv("../datasets/test_stances_unlabeled.csv", encoding='utf-8')
test = pd.merge(headline_test, body_test, how="left", on="Body ID")

In [16]:
test.shape

(25413, 3)

In [17]:
test.head(10)

,Headline,Body ID,articleBody
0,Ferguson riots: Pregnant woman loses eye after...,2008,A RESPECTED senior French police officer inves...
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,Dave Morin's social networking company Path is...
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,A bereaved Afghan mother took revenge on the T...
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,Hewlett-Packard is officially splitting in two...
4,Argentina's President Adopts Boy to End Werewo...,37,An airline passenger headed to Dallas was remo...
5,Next-generation Apple iPhones' features leaked,2353,When faced with the choice of feasting on a fi...
6,Saudi national airline may introduce gender se...,192,The US declared the video of Sotloff to be aut...
7,'Zombie Cat' Claws Way Out Of Grave And Into O...,2482,19-year-old Iga Jasica of Poland began making ...
8,"ISIS might be harvesting organs, Iraq tells UN",250,Michael Foley says the administration threaten...
9,Woman has surgery to get third breast: The thr...,85,Brian Stelter from CNN just reported that hack...


In [18]:
data = pd.concat((train, test), sort=False)

In [19]:
data.head(10)

,Headline,Body ID,Stance,articleBody,target
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3.0
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...,0.0
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...,3.0
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...,3.0
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's...",1.0
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,agree,Thousands of people have been duped by a fake ...,0.0
6,Accused Boston Marathon Bomber Severely Injure...,962,unrelated,A British fighter who travelled to Iraq to sto...,3.0
7,Identity of ISIS terrorist known as 'Jihadi Jo...,2033,unrelated,"Adding to Apple's iOS 8 launch troubles, a rep...",3.0
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree,If you’ve seen a story floating around on your...,0.0
9,British Aid Worker Confirmed Murdered By ISIS,882,unrelated,The British Islamic State militant who has fea...,3.0


In [20]:
data.shape

(75385, 5)

In [21]:
train = data[~data['target'].isnull()]
test = data[data['target'].isnull()]

In [22]:
print(train.shape, test.shape)

(49972, 5) (25413, 5)


In [23]:
print ("generate unigram")
data["Headline_unigram"] = data["Headline"].map(lambda x: preprocess_data(x))
data["articleBody_unigram"] = data["articleBody"].map(lambda x: preprocess_data(x))

generate unigram


In [24]:
print ("generate bigram")
join_str = "_"
data["Headline_bigram"] = data["Headline_unigram"].map(lambda x: ngram.getBigram(x, join_str))
data["articleBody_bigram"] = data["articleBody_unigram"].map(lambda x: ngram.getBigram(x, join_str))

generate bigram


In [25]:
print ("generate trigram")
join_str = "_"
data["Headline_trigram"] = data["Headline_unigram"].map(lambda x: ngram.getTrigram(x, join_str))
data["articleBody_trigram"] = data["articleBody_unigram"].map(lambda x: ngram.getTrigram(x, join_str))

generate trigram


In [26]:
with open('../saved_data/data.pkl', 'wb') as outfile:
    pickle.dump(data, outfile, -1)
    print('dataframe saved in data.pkl')

dataframe saved in data.pkl


In [31]:
data.head(10)

,Headline,Body ID,Stance,articleBody,target,Headline_unigram,articleBody_unigram,Headline_bigram,articleBody_bigram,Headline_trigram,articleBody_trigram
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...,3.0,"[polic, find, mass, grave, least, 15, bodi, ne...","[danni, boyl, direct, untitl, film, seth, roge...","[polic_find, find_mass, mass_grave, grave_leas...","[danni_boyl, boyl_direct, direct_untitl, untit...","[polic_find_mass, find_mass_grave, mass_grave_...","[danni_boyl_direct, boyl_direct_untitl, direct..."
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...,0.0,"[hundr, palestinian, flee, flood, gaza, israel...","[hundr, palestinian, evacu, home, sunday, morn...","[hundr_palestinian, palestinian_flee, flee_flo...","[hundr_palestinian, palestinian_evacu, evacu_h...","[hundr_palestinian_flee, palestinian_flee_floo...","[hundr_palestinian_evacu, palestinian_evacu_ho..."
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...,3.0,"[christian, bale, pass, role, steve, job, acto...","[30, year, old, moscow, resid, hospit, wound, ...","[christian_bale, bale_pass, pass_role, role_st...","[30_year, year_old, old_moscow, moscow_resid, ...","[christian_bale_pass, bale_pass_role, pass_rol...","[30_year_old, year_old_moscow, old_moscow_resi..."
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...,3.0,"[hbo, appl, talk, 15, month, appl, tv, stream,...","[reuter, canadian, soldier, shot, canadian, wa...","[hbo_appl, appl_talk, talk_15, 15_month, month...","[reuter_canadian, canadian_soldier, soldier_sh...","[hbo_appl_talk, appl_talk_15, talk_15_month, 1...","[reuter_canadian_soldier, canadian_soldier_sho..."
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's...",1.0,"[spider, burrow, tourist, stomach, chest]","[fear, arachnophob, stori, bunburi, spiderman,...","[spider_burrow, burrow_tourist, tourist_stomac...","[fear_arachnophob, arachnophob_stori, stori_bu...","[spider_burrow_tourist, burrow_tourist_stomach...","[fear_arachnophob_stori, arachnophob_stori_bun..."
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,agree,Thousands of people have been duped by a fake ...,0.0,"[nasa, confirm, earth, experi, day, total, dar...","[thousand, peopl, dupe, fake, news, stori, cla...","[nasa_confirm, confirm_earth, earth_experi, ex...","[thousand_peopl, peopl_dupe, dupe_fake, fake_n...","[nasa_confirm_earth, confirm_earth_experi, ear...","[thousand_peopl_dupe, peopl_dupe_fake, dupe_fa..."
6,Accused Boston Marathon Bomber Severely Injure...,962,unrelated,A British fighter who travelled to Iraq to sto...,3.0,"[accus, boston, marathon, bomber, sever, injur...","[british, fighter, travel, iraq, stop, islam, ...","[accus_boston, boston_marathon, marathon_bombe...","[british_fighter, fighter_travel, travel_iraq,...","[accus_boston_marathon, boston_marathon_bomber...","[british_fighter_travel, fighter_travel_iraq, ..."
7,Identity of ISIS terrorist known as 'Jihadi Jo...,2033,unrelated,"Adding to Apple's iOS 8 launch troubles, a rep...",3.0,"[ident, isi, terrorist, known, jihadi, john, r...","[ad, appl, io, launch, troubl, report, monday,...","[ident_isi, isi_terrorist, terrorist_known, kn...","[ad_appl, appl_io, io_launch, launch_troubl, t...","[ident_isi_terrorist, isi_terrorist_known, ter...","[ad_appl_io, appl_io_launch, io_launch_troubl,..."
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree,If you’ve seen a story floating around on your...,0.0,"[banksi, arrest, real, ident, reveal, hoax, la...","[seen, stori, float, around, facebook, feed, b...","[banksi_arrest, arrest_real, real_ident, ident...","[seen_stori, stori_float, float_around, around...","[banksi_arrest_real, arrest_real_ident, real_i...","[seen_stori_float, st

## Ends here, the preprocessing and pickling the data
### Next step>>> Get features.